**=================================================     
INSTRUÇÕES PARA RODAR O ARQUIVO _ONLINE_    
=================================================**    

Quem acessar o arquivo via plataforma GITHUB.IO deverá rodar o arquivo seguindo os seguintes passos:

> Clique no ícone representado por três argolas **_Execute of Binder_**  no canto superior direito da página;  
> Espere o arquivo carregar, leva aguns segundos !!!;  
> O menu de execução irá aparecer, clique em **_Kernel_** e depois em **_Restart & Run All_**;  
> Caso deseje alterar algum dos dados só clique na célula correpondente, alterar o valor e rode novamente;  
> A tabela com os resultados da otimização irá aparecer no console do notebook jupyter.  

**=================================================  
BIBLIOTECA DE FUNÇÕES  
=================================================**  

In [1]:
!pip install numpy
!pip install pandas
import numpy as np
import pandas as pd

In [2]:
def calculo_nrd_vigas(betaX, **kwargs):
    # Atribuindo valores pelo parâmetro kwargs (Parâmetros da viga)
    betaX2com3 = kwargs.get('betaX2com3')
    defStTrac = kwargs.get('defStTrac')
    defSt50 = kwargs.get('defSt50')
    defSt60 = kwargs.get('defSt60')
    modEAco = kwargs.get('modEAco')
    fydTrac = kwargs.get('fydTrac')
    fydComp = kwargs.get('fydComp')
    d = kwargs.get('d')
    dlinha = kwargs.get('dlinha')
    alfaC = kwargs.get('alfaC')
    fck = kwargs.get('fck')
    fcd = kwargs.get('fcd')
    bw = kwargs.get('bw')
    profLambda = kwargs.get('profLambda')
    asTrac = kwargs.get('asTrac')
    asComp = kwargs.get('asComp')
    # Determinação do valor da deformação do aço tracionado
    if betaX <= betaX2com3:
        defStTrac = 10/1000
        defConcre = (defStTrac*betaX)/(1 - betaX)
    else:
        if fck > 50:
            epsilonCu = 2.6/1000 + 35/1000*((90 - fck)/100)**4
            defConcre = epsilonCu
            defStTrac = (defConcre*(1 - betaX))/betaX
        else:
            epsilonCu = 3.5/1000
            defConcre = epsilonCu
            defStTrac = (defConcre*(1 - betaX))/betaX
    # Determinação do valor da tensão do aço tracionado
    if defStTrac < defSt50:
        sigSt = -modEAco*defStTrac
    else:
        sigSt = -fydTrac
    linhaNeutra = betaX*d
    # Determinação do valor da tensão do aço compr. pela tensão do concreto
    if linhaNeutra >= dlinha:
        defStComp = defConcre*((linhaNeutra - dlinha)/linhaNeutra)
    else:
        defStComp = defConcre*((dlinha - linhaNeutra)/linhaNeutra)
    # Determinação do valor da tensão do aço comprimido
    if defStComp < defSt60:
        if linhaNeutra >= dlinha:
            sigStComp = modEAco*defStComp
        else:
            sigStComp = -modEAco*defStComp
    else:
        if linhaNeutra >= dlinha:
            sigStComp = fydComp
        else:
            sigStComp = -fydComp
    # Força no concreto comprimido
    fRcc = (alfaC*fcd*bw)*(profLambda*linhaNeutra)
    # Força no aço tracionado
    fRst = sigSt*asTrac
    # Força no aço comprimido
    fRsc = sigStComp*asComp
    # Força normal resultante no sistema estrutural
    nRd = fRcc + fRsc + fRst
    return nRd, fRcc, fRsc, fRst, defStTrac, defStComp, defConcre


def metodo_da_bissecao(xMin, xMax, nIter, **kwargs):
    resultados = np.zeros((nIter, 13))
    for iteracao in range(nIter):
        # Cálculo do novo ponto no método da bisseção
        xNew = (xMin + xMax)/2
        # Armazenamento parte 1
        resultados[iteracao][0] = iteracao
        resultados[iteracao][1] = xMin
        resultados[iteracao][2] = xMax
        resultados[iteracao][3] = xNew
        nRdxNew, fRccxNew, fRscxNew, fRstxNew, defStTracNew, defStCompNew, defConcreNew = calculo_nrd_vigas(xNew, **kwargs)
        nRdxMin, fRcc, fRsc, fRst, defStTrac, defStComp, defConcre  = calculo_nrd_vigas(xMin, **kwargs)
        nRdxMax, fRcc, fRsc, fRst, defStTrac, defStComp, defConcre = calculo_nrd_vigas(xMax, **kwargs)
        # Verificação da parcela do domínio onde está registrada a possível
        # raiz da equação
        if (nRdxMin*nRdxNew) <= 0:
            xMin = xMin
            xMax = xNew
        elif (nRdxMin*nRdxNew) > 0:
            xMin = xNew
            xMax = xMax
        resultados[iteracao][4] = xMin
        resultados[iteracao][5] = xMax
        resultados[iteracao][6] = fRccxNew
        resultados[iteracao][7] = fRscxNew
        resultados[iteracao][8] = fRstxNew
        resultados[iteracao][9] = nRdxNew
        resultados[iteracao][10] = defStTracNew*1000
        resultados[iteracao][11] = defStCompNew*1000
        resultados[iteracao][12] = defConcreNew*1000
    return resultados, xNew


**=================================================  
RESOLUÇÃO DO PROBLEMA  
=================================================**  

Aqui são colocados os dados da viga de concreto conforme exercíco do professor Dr. Ricardo Carrazedo. Os dados devem ser informados para o material e para a seção transversal desejada.

In [3]:
# Dados relativos aos materiais
fck = 30
alfaC = 0.85
profLambda = 0.80
defSt50 = 2.07/1000
defSt60 = 2.48/1000
modEAco = 21000
asTrac = 5*8.00
asComp = 2*0.20
fykTrac = 500
fykComp = 600
fcd = fck/1.4/10
fydTrac = fykTrac/1.15/10
fydComp = fykComp/1.15/10

# Dados relativos a seção
h = 50
bw = 25
cob = 2.5
fiEstribo = 5
fiLongTrac = 32.0
fiLongComp = 5
 
d = h - (cob + fiEstribo/10 + fiLongTrac/2/10)
dlinha = cob + fiEstribo/10 + fiLongComp/2/10
betaX2com3 = 0.26;
betaX3com4 = 0.45;

# Dados do método numérico
betaMin = 0.00001
betaMax = d/h
nIter = 50

# Chamada do problema de otimização irrestrita
testeMesa, betaX = metodo_da_bissecao(betaMin, betaMax, nIter, betaX2com3=betaX2com3, defSt50=defSt50, modEAco=modEAco, fydTrac=fydTrac, fydComp=fydComp, d=d, dlinha=dlinha, defSt60=defSt60, alfaC=alfaC, fcd=fcd, bw=bw, profLambda=profLambda, asTrac=asTrac, asComp=asComp, fck=fck)
testeMesa = pd.DataFrame(testeMesa)
testeMesa.columns=['iter', 'BetaMin', 'BetaMax', 'Beta','NBetaMin','NBetaMax','Rcc (kN)','Rsc (KN)','Rst (kN)','Nrd (kN)','DefTrac', 'DefComp', 'DefConc']

print("Exemplo de caminho C:\\Users\\Usuario\\Documents\\Downloads\\ ..Não é necessário colocar o nome do arquivo .xlsx")
caminho = input('Insira o caminho onde vc deseja salvar: ')
testeMesa.to_excel(caminho + 'Planilha de Resultados.xlsx')


Exemplo de caminho C:\Users\Usuario\Documents\Downloads\ ..Não é necessário colocar o nome do arquivo .xlsx
Insira o caminho onde vc deseja salvar: C:\Users\Usuario\Documents\Downloads\viga2


In [4]:
testeMesa

,iter,BetaMin,BetaMax,Beta,NBetaMin,NBetaMax,Rcc (kN),Rsc (KN),Rst (kN),Nrd (kN),DefTrac,DefComp,DefConc
0,0.0,0.000010,0.908000,0.454005,0.454005,0.908000,750.859412,20.869565,-1739.130435,-9.674015e+02,4.209166,2.948132,3.5
1,1.0,0.454005,0.908000,0.681002,0.681002,0.908000,1126.280849,20.869565,-1377.164768,-2.300144e+02,1.639482,3.132086,3.5
2,2.0,0.681002,0.908000,0.794501,0.681002,0.794501,1313.991567,20.869565,-760.434707,5.744264e+02,0.905279,3.184644,3.5
3,3.0,0.681002,0.794501,0.737752,0.681002,0.737752,1220.136208,20.869565,-1045.079672,1.959261e+02,1.244142,3.160386,3.5
4,4.0,0.681002,0.737752,0.709377,0.709377,0.737752,1173.208529,20.869565,-1204.480612,-1.040252e+01,1.433905,3.146802,3.5
5,5.0,0.709377,0.737752,0.723565,0.709377,0.723565,1196.672368,20.869565,-1123.217409,9.432452e+01,1.337164,3.153727,3.5
6,6.0,0.709377,0.723565,0.716471,0.709377,0.716471,1184.940448,20.869565,-1163.446723,4.236329e+01,1.385056,3.150299,3.5
7,7.0,0.709377,0.716471,0.712924,0.709377,0.712924,1179.074488,20.869565,-1183.861594,1.608246e+01,1.409359,3.148559,3.5
8,8.0,0.709377,0.712924,0.711151,0.709377,0.711151,1176.141509,20.869565,-1194.145394,2.865680e+00,1.421602,3.147683,3.5
9,9.0,0.709377,0.711151,0.710264,0.710264,0.711151,1174.675019,20.869565,-1199.306551,-3.761968e+00,1.427746,3.147243,3.5


In [5]:
# Momento resistente
print('betaX =', betaX)
linhaNeutra = betaX*d
print('LN =', linhaNeutra, 'cm')
# Força no concreto comprimido
fRcc = testeMesa.iloc[nIter - 1, 6]
# Força no aço comprimido
fRsc = testeMesa.iloc[nIter - 1, 7]
Mrd = fRcc*(d - 0.50*profLambda*linhaNeutra) + fRsc*(d - dlinha)
print('Mrd =', Mrd, 'kN.cm')

betaX = 0.7107669964438805
LN = 32.268821638552176 cm
Mrd = 39074.782090422734 kN.cm
